In [ ]:
from collections import defaultdict

class Layer:
    UP = 1
    DOWN = 2
    
    def __init__(self, depth=0, range=0):
        self.depth = depth
        self.range = range
        self.position = 0
        self.direction = Layer.UP
        assert(range != 1)
        
    def reset(self):
        self.position = 0
        
    def tick(self):
        if self.range < 2:
            return
        if self.direction == Layer.UP:
            self.position += 1
        if self.direction == Layer.DOWN:
            self.position -= 1
        if self.position == self.range - 1:
            self.direction = Layer.DOWN
        if self.position == 0:
            self.direction = Layer.UP
            
    @staticmethod
    def parse(string):
        d, r = string.split(': ')
        return Layer(int(d), int(r))
    
    def __str__(self):
        return " ".join("[S]" if i == self.position else "[ ]" for i in range(self.range))
    

class Firewall:
    def __init__(self, layers):
        self._layers = defaultdict(Layer)
        self._depth = 0
        for layer in layers:
            self._layers[layer.depth] = layer
            self._depth = max(self._depth, layer.depth+1)
            
    def tick(self):
        for layer in self._layers.values():
            layer.tick()
            
    def reset(self):
        for layer in self._layers.values():
            layer.reset()
            
    def caught(self, position):
        layer = self._layers[position]
        return layer.position == 0 and layer.range != 0
                        
    def find_offset(self):
        self.reset()
        caught = [False for i in range(self._depth)]
        for i in range(self._depth ** 10):
            for p in reversed(range(1, self._depth)):
                caught[p] = caught[p-1] or self.caught(p)
            caught[0] = self.caught(0)
            #print("\n{}".format(i))
            #self.print(caught)
            if i >= self._depth - 1 and not caught[-1]:
                return i - self._depth + 1
            self.tick()
    
    @staticmethod
    def parse(strings):
        return Firewall([Layer.parse(line) for line in strings])
    
    def print(self, caught):
        print("\n".join("{:2}{}{}".format(i, " " if caught[i] else "*", self._layers[i]) for i in range(self._depth)))

    def __str__(self):
        return "\n".join("{} {}".format(i, self._layers[i]) for i in range(self._depth))

test_input = """0: 3
1: 2
4: 4
6: 4"""

fw = Firewall.parse(test_input.splitlines())
print(fw.find_offset())

with open('input', 'r') as f:
    fw = Firewall.parse(f)
print(fw.find_offset())

10
